# Preprocessing for SSIT of the Cardiac Dataset
Steps: <br>
Load in data<br>
Downsample <br>
Clean Data <br>
Demultiplex w/ hashtags <br>
Integrate Data <br>
Identify groups <br>
Embed in lower dimensions <br>



In [ ]:
import scanpy as sc
import scvi
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import os


: 

Load in Data

In [ ]:
# Set the path to the dataset folder
dataset_folder = "/home/formanj/scRNAseq_model/UpDownProject/Datasets/Cardiac"

# Get the list of .h5 files in the dataset folder
h5_files = [file for file in os.listdir(dataset_folder) if file.endswith('.h5')]

# Load each .h5 file into scanpy
adata_list = []
for file in h5_files:
    file_path = os.path.join(dataset_folder, file)
    adata_list.append(sc.read_10x_h5(file_path, gex_only=False))


for i in range(len(adata_list)):
    adata_list[i].var_names_make_unique()
    adata_list[i].uns['Dataset_name'] = f'Dataset {i}'
    adata_list[i].obs['batch'] = [f'batch {i}']*adata_list[i].shape[0]
    print([j for j in adata_list[i].var_names if j.endswith('-1')])

: 

Downsample

In [ ]:
for i in range(len(adata_list)):
    print(adata_list[i].shape)
    sc.pp.filter_cells(adata_list[i], min_genes=200)
    sc.pp.filter_genes(adata_list[i], min_cells=10)
    print(adata_list[i].shape)

: 

Demultiplex

In [ ]:
for i in range(len(adata_list)):
    HTO_list = [j for j in adata_list[i].var_names if 'DAY' in j]
    print(HTO_list)
    print(adata_list[i].obs)
    for j in HTO_list:
        adata_list[i].obs[j] = adata_list[i][:,j].X.toarray().flatten()
    adata_list[i] = adata_list[i][:, [k for k in adata_list[i].var_names if k not in HTO_list]]
    print(adata_list[i].obs)
    sc.external.pp.hashsolo(adata_list[i], cell_hashing_columns=HTO_list)


: 

In [ ]:
for i in range(len(adata_list)):
    adata_list[i] = adata_list[i][adata_list[i].obs['Classification'] != 'Negative', :]

: 

Clean Data

In [ ]:
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)

for i in range(len(adata_list)):
    adata_list[i].var['mt'] = adata_list[i].var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    adata_list[i].var['ribo'] = adata_list[i].var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata_list[i], qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    #sc.pl.violin(adata[i_d], ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)
    
    plt.scatter(adata_list[i].obs['n_genes_by_counts'], adata_list[i].obs['total_counts'], c=adata_list[i].obs['pct_counts_mt'])
    plt.xlabel('genes by counts')
    plt.ylabel('total number of reads')
    plt.title(adata_list[i].uns['Dataset_name'] + ' before QC')
    plt.colorbar()
    plt.show()
    
    upper_lim = np.quantile(adata_list[i].obs.n_genes_by_counts.values, .98)
    lower_lim = np.quantile(adata_list[i].obs.n_genes_by_counts.values, .02)
    print(f'{lower_lim} to {upper_lim}')
    adata_list[i] = adata_list[i][(adata_list[i].obs.n_genes_by_counts < upper_lim) & (adata_list[i].obs.n_genes_by_counts > lower_lim)]
    adata_list[i] = adata_list[i][adata_list[i].obs.pct_counts_mt < 20]
    
    plt.scatter(adata_list[i].obs['n_genes_by_counts'], adata_list[i].obs['total_counts'], c=adata_list[i].obs['pct_counts_mt'])
    plt.xlabel('genes by counts')
    plt.ylabel('total number of reads')
    plt.title(adata_list[i].uns['Dataset_name'] + ' After QC')
    plt.colorbar()
    plt.show()
    
    # remove stimulated group from data
    adata_list[i] = adata_list[i][[j for j in adata_list[i].obs_names if 'WTC' in adata_list[i].obs['Classification'][j]], :]

: 

Integration

In [ ]:
adata = sc.concat(adata_list)
sc.pp.filter_genes(adata, min_cells = 10)
adata.X = csr_matrix(adata.X)
adata.write_h5ad('combined.h5ad')

: 

In [ ]:
#%% Normilization
# Unequal waiting of genes
sc.pp.filter_genes(adata, min_cells = 100)

adata.layers['counts'] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=5000)

adata.raw = adata
#adata = adata[:, adata.var.highly_variable]
#sc.pp.regress_out(adata[i_d], ['total_counts', 'pct_counts_mt']) # regresses out effect
#sc.pp.scale(adata[i_d], max_value=10) # treats all genes the same
    

: 

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             categorical_covariate_keys=["batch"],
                             continuous_covariate_keys=['pct_counts_mt', 'total_counts', 'pct_counts_ribo'])
#torch.set_float32_matmul_precision('medium' | 'high')
model = scvi.model.SCVI(adata)
model.train() #may take a while without GPU

: 

Latent Represnetation 

In [ ]:

adata.obsm['X_scVI'] = model.get_latent_representation()
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)
adata.write_h5ad('ProcessedData.h5ad')

: 

In [ ]:
adata = sc.read_h5ad('ProcessedData.h5ad')

: 

In [ ]:

sc.pp.neighbors(adata, use_rep = 'X_scVI')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution = 0.5)
sc.pl.umap(adata, color = ['leiden', 'batch'], frameon = False)

: 

Save data

In [ ]:
data = {
    'Classification': adata.obs['Classification'],
    'UMAP X(0)': adata.obsm['X_scVI'][:, 0],
    'UMAP X(1)': adata.obsm['X_scVI'][:, 1],
    'leiden': adata.obs['leiden'],
    'batch': adata.obs['batch']
}


data = pd.DataFrame.from_dict(data)

data.to_csv('SSIT_Data_V1.csv')

: 

: 